In [1]:
import azureml.core
import pandas as pd
import numpy as np
import logging
import json

from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Workspace, Datastore, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.authentication import AzureCliAuthentication


from sklearn.externals import joblib

In [3]:
with open("./configuration/dlvr_config.json") as f:
    config = json.load(f)

workspace_name = config["workspace_name"]
resource_group = config["resource_group"]
subscription_id = config["subscription_id"]
location = config["location"]


ws = Workspace.get(
        name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,
    )

print('Workspace name: ' + ws.name, 'Azure region: ' + ws.location, 'Subscription id: ' + ws.subscription_id,'Resource group: ' + ws.resource_group, sep='\n')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BTVE66MMY to authenticate.


WARNING - Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/35cd3241-d250-4a26-93c8-ee676440a2b7', 'tenant_id': '35cd3241-d250-4a26-93c8-ee676440a2b7'}' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'.\r\nTrace ID: ecf08450-172d-4557-bad3-51ed4b911402\r\nCorrelation ID: 0af9ec6d-1ae0-4afd-b6aa-4c6afd60e129\r\nTimestamp: 2020-06-23 18:43:50Z","error_codes":[50076],"timestamp":"2020-06-23 18:43:50Z","trace_id":"ecf08450-172d-4557-bad3-51ed4b911402","correlation_id":"0af9ec6d-1ae0-4afd-b6aa-4c6afd60e129","error_uri":"https://login.microsoftonline.com/error?code=50076","suberror":"basic_action"}'
WARNING - Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/8a692374-8817-47

Interactive authentication successfully completed.


UserErrorException: UserErrorException:
	Message: You are currently logged-in to 72f988bf-86f1-41af-91ab-2d7cd011db47 tenant. You don't have access to 87eebea1-6e48-4a13-8ad5-8a89ed77fe10 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Shiva-MicrosoftAzureInternalConsumption', subscription_id='46668180-b0ad-4a49-bed9-88f16f315dce'), SubscriptionInfo(subscription_name='Microsoft Azure Internal Consumption', subscription_id='97227f43-3143-4d5f-921c-bc9de25314e2'), SubscriptionInfo(subscription_name='MSR AI - MLO', subscription_id='d20432c3-8203-4c39-865a-7c6c2fb1ffd2'), SubscriptionInfo(subscription_name='Customer 360 DevTest 001', subscription_id='7c469e39-c3bc-4752-a3f1-7447c5aecfa8'), SubscriptionInfo(subscription_name='Skype-NetEM-PROD', subscription_id='8cff5d56-95fb-4a74-ab9d-079edb45313e'), SubscriptionInfo(subscription_name='Code generate Test and Infra', subscription_id=

In [5]:
# choose a name for the run history container in the workspace
experiment_name = 'dlvr-automl-experiment'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['SKU'] = ws.sku
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Subscription ID,46668180-b0ad-4a49-bed9-88f16f315dce
Workspace,ShivaMLservice
SKU,Enterprise
Resource Group,MLGroup
Location,westus2
Run History Name,dlvr-automl-experiment


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "standard-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [8]:
datastore_name = 'dlvr_scorecard_store_june12'
datastore = Datastore.get(ws, datastore_name)
scorecard_dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'Scorecard-364*.csv')],header=False)

scorecard_df = scorecard_dataset.to_pandas_dataframe()
scorecard_df.head()

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,Column14
0,ellation-hls-vod-yospace,b54c47d547074946ba83b33087208830:akamai:android;samsung_browser,akamai,android;samsung_browser,-1,-1,0.00,0.00,87.94,0.00,0.00,17,1591893154920,11-16:32:34
1,ellation-hls-vod-yospace,b54c47d547074946ba83b33087208830:akamai:windows_br5_be5,akamai,windows_br5_be5,-1,-1,0.00,1.00,145.77,0.00,0.10,31,1591893154920,11-16:32:34
2,ellation-hls-vod-yospace,b54c47d547074946ba83b33087208830:akamai:linux_misc,akamai,linux_misc,-1,-1,0.00,1.00,280.00,0.15,0.00,55,1591893154920,11-16:32:34
3,ellation-hls-vod-yospace,b54c47d547074946ba83b33087208830:akamai:windows_br10_be5,akamai,windows_br10_be5,-1,-1,0.00,0.00,130.35,0.00,0.00,12,1591893154920,11-16:32:34
4,ellation-hls-vod-yospace,b54c47d547074946ba83b33087208830:akamai:android;chrome_mobile,akamai,android;chrome_mobile,-1,-1,0.00,2.00,66.95,0.20,0.20,5,1591893154920,11-16:32:34


In [9]:
#category = account:provider:device

# column headers
cols = ['account','category','provider','device','fragments','geo','predictorScore','hazardScore','averagePIDX',
        'averageBufferEvents','averageBitrateShifts','asn','timestamp','hrtime']

scorecard_df.columns = cols

# all features
features = ['account','timestamp', 'category','hazardScore', 'averagePIDX', 'averageBufferEvents', 'averageBitrateShifts','provider', 'device', 'asn', 'fragments', 'geo']

# get dataframe with just features
features_df = scorecard_df[features]

# convert timestamp to datetime
features_df['timestamp'] = pd.to_datetime(scorecard_df['timestamp'], unit='ms')
features_df.head()

,account,timestamp,category,hazardScore,averagePIDX,averageBufferEvents,averageBitrateShifts,provider,device,asn,fragments,geo
0,ellation-hls-vod-yospace,2020-06-11 16:32:34.920,b54c47d547074946ba83b33087208830:akamai:android;samsung_browser,0.00,87.94,0.00,0.00,akamai,android;samsung_browser,17,-1,-1
1,ellation-hls-vod-yospace,2020-06-11 16:32:34.920,b54c47d547074946ba83b33087208830:akamai:windows_br5_be5,1.00,145.77,0.00,0.10,akamai,windows_br5_be5,31,-1,-1
2,ellation-hls-vod-yospace,2020-06-11 16:32:34.920,b54c47d547074946ba83b33087208830:akamai:linux_misc,1.00,280.00,0.15,0.00,akamai,linux_misc,55,-1,-1
3,ellation-hls-vod-yospace,2020-06-11 16:32:34.920,b54c47d547074946ba83b33087208830:akamai:windows_br10_be5,0.00,130.35,0.00,0.00,akamai,windows_br10_be5,12,-1,-1
4,ellation-hls-vod-yospace,2020-06-11 16:32:34.920,b54c47d547074946ba83b33087208830:akamai:android;chrome_mobile,2.00,66.95,0.20,0.20,akamai,android;chrome_mobile,5,-1,-1


In [12]:
features_df.shape

(150241, 12)

In [13]:
features_df['account'].value_counts()

wb-desktop-hls-vod          32905
funimation-hls-vod          28811
ellation-hls-vod            21516
wb-seg-only-hls-vod         18976
comcast-nbcgolf-hls-vod     14895
reuters-dlvr-hls-live       12254
vevo-hls-vod                10402
comcast-amc-hls-dfp-vod     7703 
telia-digita-hls-live       1653 
ellation-hls-vod-yospace    791  
ellation-qa-hls-vod         294  
ellation-proto-hls-vod      41   
Name: account, dtype: int64

In [18]:
account_name = 'wb-desktop-hls-vod'
account_df = features_df[features_df['account'] == account_name]
account_df.head()

,account,timestamp,category,hazardScore,averagePIDX,averageBufferEvents,averageBitrateShifts,provider,device,asn,fragments,geo
315,wb-desktop-hls-vod,2020-06-11 16:37:15.518,3aa37dc0e8bb47e08042e0ebb25acb34:level3:windows;trontv,0.00,75.23,0.00,0.00,level3,windows;trontv,37,-1,-1
316,wb-desktop-hls-vod,2020-06-11 16:37:15.518,3aa37dc0e8bb47e08042e0ebb25acb34:level3:ps4_software;netfront,0.00,240.86,0.01,0.00,level3,ps4_software;netfront,234,-1,-1
317,wb-desktop-hls-vod,2020-06-11 16:37:15.518,3aa37dc0e8bb47e08042e0ebb25acb34:comcast:android;duckduckgo_for_android,0.00,749.30,0.00,0.00,comcast,android;duckduckgo_for_android,18,-1,-1
318,wb-desktop-hls-vod,2020-06-11 16:37:15.518,3aa37dc0e8bb47e08042e0ebb25acb34:level3:linux_misc,0.00,0.00,0.00,0.00,level3,linux_misc,3,-1,-1
319,wb-desktop-hls-vod,2020-06-11 16:37:15.518,3aa37dc0e8bb47e08042e0ebb25acb34:onnet:ubuntu;firefox,0.00,0.00,0.00,0.00,onnet,ubuntu;firefox,10,-1,-1


In [19]:
time_column_name = 'timestamp'
grain_column_names = ['category']
nseries = account_df.groupby(grain_column_names).ngroups
print('Data contains {0} individual time-series.'.format(nseries))

Data contains 164 individual time-series.


In [23]:
use_subset = ['3aa37dc0e8bb47e08042e0ebb25acb34:level3:windows;trontv', '3aa37dc0e8bb47e08042e0ebb25acb34:level3:ps4_software;netfront']
data_subset = account_df[account_df.category.isin(use_subset)]
nseries = data_subset.groupby(grain_column_names).ngroups
print('Data subset contains {0} individual time-series.'.format(nseries))

Data subset contains 2 individual time-series.


In [25]:
account_df['category'].value_counts()

3aa37dc0e8bb47e08042e0ebb25acb34:level3:ipados;apple_core_media                                331
3aa37dc0e8bb47e08042e0ebb25acb34:comcast:linux;chrome                                          331
3aa37dc0e8bb47e08042e0ebb25acb34:comcast:windows_br5_be5                                       331
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:ipados;apple_core_media                                331
3aa37dc0e8bb47e08042e0ebb25acb34:level3:tizen;samsung_browser                                  331
3aa37dc0e8bb47e08042e0ebb25acb34:comcast:macos_chrome                                          331
3aa37dc0e8bb47e08042e0ebb25acb34:onnet:ps4_software;netfront                                   331
3aa37dc0e8bb47e08042e0ebb25acb34:comcast:macos_ffox                                            331
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:windows_br5_be5                                        331
3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5                                         331
3aa37dc0e8

In [27]:
sum(account_df['category'].value_counts() == 331)

51

In [62]:
#filtering on top categories
data_subset = account_df[account_df['category'].isin(account_df['category'].value_counts()[account_df['category'].value_counts() == 331].index)]
nseries = data_subset.groupby(grain_column_names).ngroups
print('Data subset contains {0} individual time-series.'.format(nseries))

Data subset contains 51 individual time-series.


In [88]:
data_subset.shape

(16881, 12)

In [70]:
data_subset.index = data_subset[time_column_name]
data_subset.index

DatetimeIndex(['2020-06-11 16:37:15.518000', '2020-06-11 16:37:15.518000',
               '2020-06-11 16:37:15.518000', '2020-06-11 16:37:15.518000',
               '2020-06-11 16:37:15.518000', '2020-06-11 16:37:15.518000',
               '2020-06-11 16:37:15.518000', '2020-06-11 16:37:15.518000',
               '2020-06-11 16:37:15.518000', '2020-06-11 16:37:15.518000',
               ...
               '2020-06-12 02:23:49.807000', '2020-06-12 02:23:49.807000',
               '2020-06-12 02:23:49.807000', '2020-06-12 02:23:49.807000',
               '2020-06-12 02:23:49.807000', '2020-06-12 02:23:49.807000',
               '2020-06-12 02:23:49.807000', '2020-06-12 02:23:49.807000',
               '2020-06-12 02:23:49.807000', '2020-06-12 02:23:49.807000'],
              dtype='datetime64[ns]', name='timestamp', length=16881, freq=None)

In [90]:
data_subset.groupby('category').resample('T', how='mean')

hazardScore  \
category                                                      timestamp                          
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile 2020-06-11 16:37:00 0.00           
                                                              2020-06-11 16:38:00 0.00           
                                                              2020-06-11 16:39:00 0.00           
                                                              2020-06-11 16:40:00 nan            
                                                              2020-06-11 16:41:00 nan            
                                                              2020-06-11 16:42:00 nan            
                                                              2020-06-11 16:43:00 nan            
                                                              2020-06-11 16:44:00 0.00           
                                                              2020-06-11 16:45:00 nan            
                                                              2020-06-11 16:46:00 nan            
                                                              2020-06-11 16:47:00 0.00           
                                                              2020-06-11 16:48:00 nan            
                                                              2020-06-11 16:49:00 nan            
                                                              2020-06-11 16:50:00 nan            
                                                              2020-06-11 16:51:00 nan            
                                                              2020-06-11 16:52:00 nan            
                                                              2020-06-11 16:53:00 nan            
                                                              2020-06-11 16:54:00 nan            
                                                              2020-06-11 16:55:00 nan            
                                                              2020-06-11 16:56:00 nan            
                                                              2020-06-11 16:57:00 nan            
                                                              2020-06-11 16:58:00 nan            
                                                              2020-06-11 16:59:00 nan            
                                                              2020-06-11 17:00:00 0.00           
                                                              2020-06-11 17:01:00 nan            
                                                              2020-06-11 17:02:00 nan            
                                                              2020-06-11 17:03:00 nan            
                                                              2020-06-11 17:04:00 nan            
                                                              2020-06-11 17:05:00 nan            
                                                              2020-06-11 17:06:00 nan            
...                                                                                ..            
3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5        2020-06-12 01:54:00 0.00           
                                                              2020-06-12 01:55:00 0.00           
                                                              2020-06-12 01:56:00 nan            
                                                              2020-06-12 01:57:00 nan            
                                                              2020-06-12 01:58:00 0.00           
                                                              2020-06-12 01:59:00 nan            
                                                              2020-06-12 02:00:00 nan            
                                                              2020-06-12 02:01:00 nan            
                                                              202

In [95]:
data_subset.groupby('category').resample('3min', how='mean')

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:1502: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  return get_resampler_for_grouping(self, rule, *args, **kwargs)


hazardScore  \
category                                                      timestamp                          
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile 2020-06-11 16:36:00 0.00           
                                                              2020-06-11 16:39:00 0.00           
                                                              2020-06-11 16:42:00 0.00           
                                                              2020-06-11 16:45:00 0.00           
                                                              2020-06-11 16:48:00 nan            
                                                              2020-06-11 16:51:00 nan            
                                                              2020-06-11 16:54:00 nan            
                                                              2020-06-11 16:57:00 nan            
                                                              2020-06-11 17:00:00 0.00           
                                                              2020-06-11 17:03:00 nan            
                                                              2020-06-11 17:06:00 0.00           
                                                              2020-06-11 17:09:00 0.00           
                                                              2020-06-11 17:12:00 0.00           
                                                              2020-06-11 17:15:00 0.00           
                                                              2020-06-11 17:18:00 nan            
                                                              2020-06-11 17:21:00 nan            
                                                              2020-06-11 17:24:00 0.00           
                                                              2020-06-11 17:27:00 nan            
                                                              2020-06-11 17:30:00 0.00           
                                                              2020-06-11 17:33:00 0.00           
                                                              2020-06-11 17:36:00 0.00           
                                                              2020-06-11 17:39:00 0.00           
                                                              2020-06-11 17:42:00 0.00           
                                                              2020-06-11 17:45:00 0.00           
                                                              2020-06-11 17:48:00 0.00           
                                                              2020-06-11 17:51:00 0.00           
                                                              2020-06-11 17:54:00 0.00           
                                                              2020-06-11 17:57:00 0.00           
                                                              2020-06-11 18:00:00 0.00           
                                                              2020-06-11 18:03:00 0.00           
...                                                                                ...           
3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5        2020-06-12 00:54:00 0.00           
                                                              2020-06-12 00:57:00 0.00           
                                                              2020-06-12 01:00:00 0.00           
                                                              2020-06-12 01:03:00 0.00           
                                                              2020-06-12 01:06:00 0.00           
                                                              2020-06-12 01:09:00 0.00           
                                                              2020-06-12 01:12:00 0.00           
                                                              2020-06-12 01:15:00 0.00           
                                                              202

In [92]:
data_subset.groupby('category').resample('5min', how='mean')

hazardScore  \
category                                                      timestamp                          
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile 2020-06-11 16:35:00 0.00           
                                                              2020-06-11 16:40:00 0.00           
                                                              2020-06-11 16:45:00 0.00           
                                                              2020-06-11 16:50:00 nan            
                                                              2020-06-11 16:55:00 nan            
                                                              2020-06-11 17:00:00 0.00           
                                                              2020-06-11 17:05:00 0.00           
                                                              2020-06-11 17:10:00 0.00           
                                                              2020-06-11 17:15:00 0.00           
                                                              2020-06-11 17:20:00 nan            
                                                              2020-06-11 17:25:00 0.00           
                                                              2020-06-11 17:30:00 0.00           
                                                              2020-06-11 17:35:00 0.00           
                                                              2020-06-11 17:40:00 0.00           
                                                              2020-06-11 17:45:00 0.00           
                                                              2020-06-11 17:50:00 0.00           
                                                              2020-06-11 17:55:00 0.00           
                                                              2020-06-11 18:00:00 0.00           
                                                              2020-06-11 18:05:00 1.00           
                                                              2020-06-11 18:10:00 1.00           
                                                              2020-06-11 18:15:00 1.00           
                                                              2020-06-11 18:20:00 1.00           
                                                              2020-06-11 18:25:00 1.00           
                                                              2020-06-11 18:30:00 1.00           
                                                              2020-06-11 18:35:00 1.00           
                                                              2020-06-11 18:40:00 0.33           
                                                              2020-06-11 18:45:00 0.00           
                                                              2020-06-11 18:50:00 0.00           
                                                              2020-06-11 18:55:00 0.00           
                                                              2020-06-11 19:00:00 0.00           
...                                                                                ...           
3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5        2020-06-11 23:55:00 0.00           
                                                              2020-06-12 00:00:00 0.00           
                                                              2020-06-12 00:05:00 0.00           
                                                              2020-06-12 00:10:00 0.00           
                                                              2020-06-12 00:15:00 0.00           
                                                              2020-06-12 00:20:00 0.00           
                                                              2020-06-12 00:25:00 0.00           
                                                              2020-06-12 00:30:00 0.00           
                                                              202

In [98]:
data_subset.groupby('category').resample('1min', how='mean', fill_method='ffill')

hazardScore  \
category                                                      timestamp                          
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile 2020-06-11 16:37:00 0.00           
                                                              2020-06-11 16:38:00 0.00           
                                                              2020-06-11 16:39:00 0.00           
                                                              2020-06-11 16:40:00 0.00           
                                                              2020-06-11 16:41:00 0.00           
                                                              2020-06-11 16:42:00 0.00           
                                                              2020-06-11 16:43:00 0.00           
                                                              2020-06-11 16:44:00 0.00           
                                                              2020-06-11 16:45:00 0.00           
                                                              2020-06-11 16:46:00 0.00           
                                                              2020-06-11 16:47:00 0.00           
                                                              2020-06-11 16:48:00 0.00           
                                                              2020-06-11 16:49:00 0.00           
                                                              2020-06-11 16:50:00 0.00           
                                                              2020-06-11 16:51:00 0.00           
                                                              2020-06-11 16:52:00 0.00           
                                                              2020-06-11 16:53:00 0.00           
                                                              2020-06-11 16:54:00 0.00           
                                                              2020-06-11 16:55:00 0.00           
                                                              2020-06-11 16:56:00 0.00           
                                                              2020-06-11 16:57:00 0.00           
                                                              2020-06-11 16:58:00 0.00           
                                                              2020-06-11 16:59:00 0.00           
                                                              2020-06-11 17:00:00 0.00           
                                                              2020-06-11 17:01:00 0.00           
                                                              2020-06-11 17:02:00 0.00           
                                                              2020-06-11 17:03:00 0.00           
                                                              2020-06-11 17:04:00 0.00           
                                                              2020-06-11 17:05:00 0.00           
                                                              2020-06-11 17:06:00 0.00           
...                                                                                ...           
3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5        2020-06-12 01:54:00 0.00           
                                                              2020-06-12 01:55:00 0.00           
                                                              2020-06-12 01:56:00 0.00           
                                                              2020-06-12 01:57:00 0.00           
                                                              2020-06-12 01:58:00 0.00           
                                                              2020-06-12 01:59:00 0.00           
                                                              2020-06-12 02:00:00 0.00           
                                                              2020-06-12 02:01:00 0.00           
                                                              202

In [111]:
final_data=data_subset.groupby('category').resample('1min', how='mean', fill_method='ffill')
final_data['category'] = final_data.index.get_level_values('category')
final_data['timestamp'] = final_data.index.get_level_values('timestamp')
final_data

hazardScore  \
category                                                      timestamp                          
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile 2020-06-11 16:37:00 0.00           
                                                              2020-06-11 16:38:00 0.00           
                                                              2020-06-11 16:39:00 0.00           
                                                              2020-06-11 16:40:00 0.00           
                                                              2020-06-11 16:41:00 0.00           
                                                              2020-06-11 16:42:00 0.00           
                                                              2020-06-11 16:43:00 0.00           
                                                              2020-06-11 16:44:00 0.00           
                                                              2020-06-11 16:45:00 0.00           
                                                              2020-06-11 16:46:00 0.00           
                                                              2020-06-11 16:47:00 0.00           
                                                              2020-06-11 16:48:00 0.00           
                                                              2020-06-11 16:49:00 0.00           
                                                              2020-06-11 16:50:00 0.00           
                                                              2020-06-11 16:51:00 0.00           
                                                              2020-06-11 16:52:00 0.00           
                                                              2020-06-11 16:53:00 0.00           
                                                              2020-06-11 16:54:00 0.00           
                                                              2020-06-11 16:55:00 0.00           
                                                              2020-06-11 16:56:00 0.00           
                                                              2020-06-11 16:57:00 0.00           
                                                              2020-06-11 16:58:00 0.00           
                                                              2020-06-11 16:59:00 0.00           
                                                              2020-06-11 17:00:00 0.00           
                                                              2020-06-11 17:01:00 0.00           
                                                              2020-06-11 17:02:00 0.00           
                                                              2020-06-11 17:03:00 0.00           
                                                              2020-06-11 17:04:00 0.00           
                                                              2020-06-11 17:05:00 0.00           
                                                              2020-06-11 17:06:00 0.00           
...                                                                                ...           
3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5        2020-06-12 01:54:00 0.00           
                                                              2020-06-12 01:55:00 0.00           
                                                              2020-06-12 01:56:00 0.00           
                                                              2020-06-12 01:57:00 0.00           
                                                              2020-06-12 01:58:00 0.00           
                                                              2020-06-12 01:59:00 0.00           
                                                              2020-06-12 02:00:00 0.00           
                                                              2020-06-12 02:01:00 0.00           
                                                              202

In [119]:
final_data = final_data.reset_index(drop=True)
final_data.head()

,hazardScore,averagePIDX,averageBufferEvents,averageBitrateShifts,asn,fragments,geo,category,timestamp
0,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:37:00
1,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:38:00
2,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:39:00
3,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:40:00
4,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:41:00


In [132]:
final_data.groupby('category').count()

,hazardScore,averagePIDX,averageBufferEvents,averageBitrateShifts,asn,fragments,geo,timestamp
category,,,,,,,,
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,587,587,587,587,587,587,587,587
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:chromeos;chrome,587,587,587,587,587,587,587,587
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:ios;apple_core_media,587,587,587,587,587,587,587,587
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:ipados;apple_core_media,587,587,587,587,587,587,587,587
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:linux;chrome,587,587,587,587,587,587,587,587
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:macos_chrome,587,587,587,587,587,587,587,587
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:macos_core,587,587,587,587,587,587,587,587
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:macos_ffox,587,587,587,587,587,587,587,587
3aa37dc0e8bb47e08042e0ebb25acb34:akamai:macos_safari,587,587,587,587,587,587,587,587


In [135]:
final_data[final_data['category'] == '3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile']

,hazardScore,averagePIDX,averageBufferEvents,averageBitrateShifts,asn,fragments,geo,category,timestamp
0,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:37:00
1,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:38:00
2,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:39:00
3,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:40:00
4,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:41:00
5,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:42:00
6,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:43:00
7,0.00,142.48,0.03,0.04,95.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:44:00
8,0.00,142.48,0.03,0.04,95.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:45:00
9,0.00,142.48,0.03,0.04,95.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:46:00


In [137]:
n_test_periods = 10

def split_last_n_by_grain(df, n):
    """Group df by grain and split on last n rows for each group."""
    df_grouped = (df.sort_values(time_column_name) # Sort by ascending time
                  .groupby(grain_column_names, group_keys=False))
    df_head = df_grouped.apply(lambda dfg: dfg.iloc[:-n])
    df_tail = df_grouped.apply(lambda dfg: dfg.iloc[-n:])
    return df_head, df_tail

train, test = split_last_n_by_grain(final_data, n_test_periods)

In [138]:
train[train['category'] == '3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile']

,hazardScore,averagePIDX,averageBufferEvents,averageBitrateShifts,asn,fragments,geo,category,timestamp
0,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:37:00
1,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:38:00
2,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:39:00
3,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:40:00
4,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:41:00
5,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:42:00
6,0.00,144.16,0.03,0.04,105.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:43:00
7,0.00,142.48,0.03,0.04,95.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:44:00
8,0.00,142.48,0.03,0.04,95.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:45:00
9,0.00,142.48,0.03,0.04,95.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:akamai:android;chrome_mobile,2020-06-11 16:46:00


In [139]:
train.to_csv (r'./scorecard_train.csv', index = None, header=True)
test.to_csv (r'./scorecard_test.csv', index = None, header=True)

In [140]:
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./scorecard_train.csv', './scorecard_test.csv'], target_path = 'scorecard_dlvr/', overwrite = True,show_progress = True)

Uploading an estimated of 2 files
Uploading ./scorecard_train.csv
Uploading ./scorecard_test.csv
Uploaded ./scorecard_test.csv, 1 files out of an estimated total of 2
Uploaded ./scorecard_train.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_54d623269a5d40b1bc2641e5350ff9a5

In [141]:
from azureml.core.dataset import Dataset
train_dataset = Dataset.Tabular.from_delimited_files(path=datastore.path('scorecard_dlvr/scorecard_train.csv'))
test_dataset = Dataset.Tabular.from_delimited_files(path=datastore.path('scorecard_dlvr/scorecard_test.csv'))


In [142]:
train_dataset = train_dataset.register(workspace = ws,
                           name = 'scorecard_train',
                           description='train dataset for dlvr',
                           create_new_version=True)

test_dataset = test_dataset.register(workspace = ws,
                           name = 'scorecard_test',
                           description='test dataset for dlvr',
                           create_new_version=True)


In [143]:
train_dataset.to_pandas_dataframe().tail()

,hazardScore,averagePIDX,averageBufferEvents,averageBitrateShifts,asn,fragments,geo,category,timestamp
29422,0.00,162.65,0.01,0.01,4971.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5,2020-06-12 02:09:00
29423,0.00,162.65,0.01,0.01,4971.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5,2020-06-12 02:10:00
29424,0.00,162.65,0.01,0.01,4971.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5,2020-06-12 02:11:00
29425,0.00,162.65,0.01,0.01,4971.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5,2020-06-12 02:12:00
29426,0.00,162.65,0.01,0.01,4971.00,-1.00,-1.00,3aa37dc0e8bb47e08042e0ebb25acb34:onnet:windows_br5_be5,2020-06-12 02:13:00
